In [1]:
%load_ext autoreload
# %autoreload 2

In [2]:
import logging
from sys import stdout
from datetime import datetime
import numpy as np
import os

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(levelname)s] %(message)s",
    stream=stdout,
    datefmt="%m-%d %H:%M:%S",
)

logging.info(f"{os.getlogin()}| {datetime.now()}")

[08-07 13:57:59][INFO] DELL| 2025-08-07 13:57:59.332931


In [3]:
# ####################################################################
def gauss_jordan(Ab: np.ndarray) -> np.ndarray:
    """Resuelve un sistema de ecuaciones lineales mediante el método de Gauss-Jordan.

    ## Parameters

    ``Ab``: matriz aumentada del sistema de ecuaciones lineales. Debe ser de tamaño n-by-(n+1), donde n es el número de incógnitas.

    ## Return

    ``solucion``: vector con la solución del sistema de ecuaciones lineales.

    """
    if not isinstance(Ab, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        Ab = np.array(Ab, dtype=float)
    assert Ab.shape[0] == Ab.shape[1] - 1, "La matriz A debe ser de tamaño n-by-(n+1)."
    n = Ab.shape[0]

    for i in range(0, n):  # loop por columna

        # --- encontrar pivote
        p = None  # default, first element
        for pi in range(i, n):
            if Ab[pi, i] == 0:
                # must be nonzero
                continue

            if p is None:
                # first nonzero element
                p = pi
                continue

            if abs(Ab[pi, i]) < abs(Ab[p, i]):
                p = pi

        if p is None:
            # no pivot found.
            raise ValueError("No existe solución única.")

        if p != i:
            # swap rows
            logging.debug(f"Intercambiando filas {i} y {p}")
            _aux = Ab[i, :].copy()
            Ab[i, :] = Ab[p, :].copy()
            Ab[p, :] = _aux

        # --- Eliminación: loop por fila
        for j in range(n):
            if i == j:
                continue
            m = Ab[j, i] / Ab[i, i]
            Ab[j, i:] = Ab[j, i:] - m * Ab[i, i:]

        logging.info(f"\n{Ab}")

    if Ab[n - 1, n - 1] == 0:
        raise ValueError("No existe solución única.")

    # --- Sustitución hacia atrás
    solucion = np.zeros(n)

    for i in range(n - 1, -1, -1):
        solucion[i] = Ab[i, -1] / Ab[i, i]

    return solucion

In [4]:
%autoreload 2
from src import matriz_aumentada

A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
b = [1, -2, 3, 4]

Ab = matriz_aumentada(A, b)
gauss_jordan(Ab)

[08-07 13:57:59][INFO] 2025-08-07 13:57:59.392973
[08-07 13:57:59][INFO] DELL| 2025-08-07 13:57:59.399870
[08-07 13:57:59][INFO] 2025-08-07 13:57:59.405965
[08-07 13:57:59][INFO] DELL| 2025-08-07 13:57:59.413289
[08-07 13:57:59][INFO] 
[[ 1.  2.  3.  4.  1.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0. -1. -3. -6.  0.]]
[08-07 13:57:59][INFO] 
[[ 1.  0.  3.  6.  9.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0.  0. -3. -7. -4.]]
[08-07 13:57:59][INFO] 
[[ 1.  0.  0. -3.  9.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0.  0.  0.  2. -4.]]
[08-07 13:57:59][INFO] 
[[ 1.  0.  0.  0.  3.]
 [ 0.  1.  0.  0. -6.]
 [ 0.  0. -1.  0. -6.]
 [ 0.  0.  0.  2. -4.]]


array([ 3., -6.,  6., -2.])

# Resolver

In [14]:
import numpy as np

def inv_matrix(A: np.ndarray) -> np.ndarray:
    """Inversión de una matriz cuadrada mediante método de Gauss-Jordan."""
    n = A.shape[0]
    
    # Crear matriz aumentada [A | I]
    I = np.identity(n)
    AI = np.hstack((A.astype(float), I))

    # Aplicar Gauss-Jordan para llevar A a identidad
    for i in range(n):
        # Si el pivote es cero, intercambiar con fila inferior
        if AI[i, i] == 0:
            for k in range(i + 1, n):
                if AI[k, i] != 0:
                    AI[[i, k]] = AI[[k, i]]  # Intercambio de filas
                    break
            else:
                raise ValueError("La matriz es singular y no tiene inversa.")
        
        # Normalizar fila i
        AI[i] = AI[i] / AI[i, i]
        
        # Hacer ceros en las demás filas
        for j in range(n):
            if j != i:
                factor = AI[j, i]
                AI[j] = AI[j] - factor * AI[i]
    
    # Extraer la parte derecha como la inversa
    A_inv = AI[:, n:]
    return A_inv


## Ejemplos
* Ejemplo 1

In [18]:
import numpy as np

A = np.array([
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10]
])

A_inv = inv_matrix(A)
print("A⁻¹ =")
print(A_inv)


A⁻¹ =
[[ 0.5 -0.5 -1.5  1.5]
 [-0.5  1.5 -1.5  0.5]
 [-1.5 -1.5  3.5 -1.5]
 [ 1.5  0.5 -1.5  0.5]]


* Ejemplo 2

In [16]:
# La matriz A =
A = [
    [4, 4, 5, 1],
    [3, 4, 2, 2],
    [2, 1, 4, 1],
    [3, 2, 5, 4],
]
# tiene como inversa
# A_inv =[[-34.,  31.,  52., -20.],
#         [ 19., -17., -29.,  11.],
#         [ 12., -11., -18.,   7.],
#         [  1.,  -1.,  -2.,   1.]]
inv_matrix(A)

AttributeError: 'list' object has no attribute 'shape'

## Ejercicios

* Ejercicio 1

In [21]:
import numpy as np

A = [[2, -3], [-1, 1]]
A_inv = inv_matrix(A)
print("A⁻¹ =")
print(A_inv)


AttributeError: 'list' object has no attribute 'shape'

* Ejercicio 2

In [9]:
A = [
    [4, 0, 0, 5],
    [1, 0, 4, 0],
    [3, 4, 1, 3],
    [1, 3, 3, 0],
]
inv_matrix(A)

* Ejercicio 3

In [10]:
A = [
    [0, 0, 0, 0, 0, 0, 1, -1],
    [0, 1, -1, 1, 0, -1, 0, 1],
    [-1, -1, 0, 0, 2, 1, 0, 0],
    [-1, -1, -1, 1, 2, 0, 0, 1],
    [-1, 1, 1, 0, -1, -1, 0, 2],
    [0, 1, 0, 0, -1, -1, 0, 0],
    [1, -1, -1, 1, 2, 1, 0, 2],
    [2, 0, 0, 0, 0, 1, 2, 0],
]
inv_matrix(A)

* Ejercicio 4

In [11]:
A = [
    [1, 0, 0, 0, -1, 0, 0, -1, 1, -1],
    [1, 1, 0, -1, -1, 1, 0, 0, 1, -1],
    [-1, 0, -1, 0, 0, 0, -1, 1, 0, 0],
    [0, 0, -1, 0, -1, -1, 1, 0, 1, 0],
    [-1, 0, 0, -1, 1, 1, 1, 1, 0, -1],
    [1, 0, 0, 1, -1, -1, -1, 1, -1, 0],
    [1, 1, 1, 0, 1, 0, -1, -1, -1, 1],
    [1, 1, 1, 1, 0, 0, 1, 1, 0, 0],
    [1, 1, 1, 1, 1, 0, -1, -1, 0, 0],
    [0, 0, -1, -1, -1, 0, 1, 1, 1, -1],
]
inv_matrix(A)